## Preprocessing the Credit Card Transactions data

In [1]:
data_path = "D:\\Users\\r0625260\\Downloads\\wetransfer-9c05d3\\Credit Card Transaction Data.csv"

In [2]:
# Import the credit card dataset (decimal separator needs to be changed from ',' to '.', csv separator is ';')
import pandas as pd
df = pd.read_csv(data_path, sep=';', index_col=0, parse_dates=[6], decimal=',')

D:\Users\r0625260\.conda\envs\newenv\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


`CARD_PAN_ID` and `TERM_MIDUID` will be used to identify client and merchant nodes respectively. We replace these string values with integers and ensure that client, transaction and merchant nodes have globally unique ID's.

In [6]:
node_identifier1 = pd.DataFrame(df['CARD_PAN_ID'].unique())
node_identifier1['numeric_node_identifier1'] = node_identifier1.index + max(df.index) + 1

node_identifier2 = pd.DataFrame(df['TERM_MIDUID'].unique())
node_identifier2['numeric_node_identifier2'] = node_identifier2.index + max(node_identifier1['numeric_node_identifier1']) + 1
def update_type(t1, t2, dropna=False):
    return t1.map(t2).dropna() if dropna else t1.map(t2).fillna(t1)
updated = update_type(df.CARD_PAN_ID, node_identifier1.set_index(0).numeric_node_identifier1)

df = pd.concat([df.drop('CARD_PAN_ID', axis = 1), updated], axis=1)
updated = update_type(df.TERM_MIDUID, node_identifier2.set_index(0).numeric_node_identifier2)

df = pd.concat([df.drop('TERM_MIDUID', axis = 1), updated], axis=1)


This cell transforms the remaining input dataset into numeric format. We transform the `TX_FRAUD` and `TX_ACCEPTED` labels from boolean to binary values. In addition, the timestamp is converted into its components, and dummy variables are created for the country attribute. 

In [ ]:
df.TX_FRAUD = df.TX_FRAUD.astype(int)

df.TX_ACCEPTED = df.TX_ACCEPTED.astype(int)

df['year'] = df.TX_DATETIME.dt.year
df['month'] = df.TX_DATETIME.dt.month
df['day'] = df.TX_DATETIME.dt.day
df['hour'] = df.TX_DATETIME.dt.hour
df['minutes'] = df.TX_DATETIME.dt.minute
df['seconds'] = df.TX_DATETIME.dt.second
df = df.drop('TX_DATETIME', axis=1)
df = pd.get_dummies(df)
df.apply(pd.to_numeric) 